In [11]:
!pip install openpyxl

In [12]:
import pandas as pd
import io
import os

# --- CONFIGURATION ---
# Since we are working locally in SageMaker, we set this to False
IS_AWS = False 
input_file = "ILIDataV2.xlsx"
years = ["2007", "2015", "2022"]

mappings = {
    "2007": {"log dist. [ft]": "distance", "o'clock": "clock", "event": "feature_type", "depth [%]": "depth", "length [in]": "length", "width [in]": "width"},
    "2015": {"Log Dist. [ft]": "distance", "O'clock": "clock", "Event Description": "feature_type", "Depth [%]": "depth", "Length [in]": "length", "Width [in]": "width"},
    "2022": {"ILI Wheel Count \n[ft.]": "distance", "O'clock\n[hh:mm]": "clock", "Event Description": "feature_type", "Metal Loss Depth \n[%]": "depth", "Length [in]": "length", "Width [in]": "width"}
}

def run_standardization():
    try:
        # 1. LOAD DATA Local mode
        print(f"🐧 Opening {input_file}...")
        with open(input_file, "rb") as f:
            excel_data = f.read()

        for year in years:
            # Read sheet
            df = pd.read_excel(io.BytesIO(excel_data), sheet_name=year)
            
            # Apply Gia's mappings
            df = df.rename(columns=mappings[year])
            df['survey_year'] = int(year)
            
            # Keep only the columns needed for alignment
            target_cols = ["distance", "clock", "feature_type", "depth", "length", "width", "survey_year"]
            existing_cols = [c for c in target_cols if c in df.columns]
            df_final = df[existing_cols]

            # 2. SAVE DATA locally
            output_folder = "standardized"
            output_filename = f"ILI_{year}_cleaned.csv"
            
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            
            df_final.to_csv(os.path.join(output_folder, output_filename), index=False)
            print(f"✅ Processed {year} successfully. Saved to {output_folder}/{output_filename}")

        return "Standardization complete."

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return str(e)

# Run the process
run_standardization()

🐧 Opening ILIDataV2.xlsx...
✅ Processed 2007 successfully. Saved to standardized/ILI_2007_cleaned.csv
✅ Processed 2015 successfully. Saved to standardized/ILI_2015_cleaned.csv
✅ Processed 2022 successfully. Saved to standardized/ILI_2022_cleaned.csv


'Standardization complete.'

In [13]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Load the standardized CSVs
df_07 = pd.read_csv('standardized/ILI_2007_cleaned.csv')
df_15 = pd.read_csv('standardized/ILI_2015_cleaned.csv')
df_22 = pd.read_csv('standardized/ILI_2022_cleaned.csv')

def align_with_master_anchor(df_source, df_ref):
    # 1. Find the first Valve in both (This is our "True Zero")
    anchor_src_val = df_source[df_source['feature_type'].str.contains('Valve', na=False)]['distance'].iloc[0]
    anchor_ref_val = df_ref[df_ref['feature_type'].str.contains('Valve', na=False)]['distance'].iloc[0]
    
    # 2. Filter Girth Welds that appear AFTER the valve
    anchors_src = df_source[(df_source['feature_type'].str.contains('Girth', na=False)) & 
                            (df_source['distance'] > anchor_src_val)]
    anchors_ref = df_ref[(df_ref['feature_type'].str.contains('Girth', na=False)) & 
                         (df_ref['distance'] > anchor_ref_val)]
    
    # 3. Create staple points (adding the Valve at the start)
    old_x = [anchor_src_val] + anchors_src['distance'].tolist()
    new_x = [anchor_ref_val] + anchors_ref['distance'].tolist()
    
    # Match indices
    n = min(len(old_x), len(new_x))
    transform = interp1d(old_x[:n], new_x[:n], fill_value="extrapolate")
    
    # 4. Transform the distances
    df_source['aligned_dist'] = transform(df_source['distance'])
    return df_source

# Run the improved alignment
print("🐧 Re-aligning with Valve-First Strategy...")
df_07_final = align_with_master_anchor(df_07, df_22)
df_15_final = align_with_master_anchor(df_15, df_22)

# --- NEW SANITY CHECK ---
v_22 = df_22[df_22['feature_type'].str.contains('Valve', na=False)]['distance'].iloc[0]
v_07_aligned = df_07_final[df_07_final['feature_type'].str.contains('Valve', na=False)]['aligned_dist'].iloc[0]

print(f"\n--- Improved Valve Sanity Check ---")
print(f"2022 Distance: {v_22:.4f} ft")
print(f"2007 Aligned:  {v_07_aligned:.4f} ft")
print(f"New Error:     {abs(v_22 - v_07_aligned):.4f} ft (Target: < 0.1)")

# Save the final mapped files
df_07_final.to_csv('standardized/ILI_2007_aligned.csv', index=False)
df_15_final.to_csv('standardized/ILI_2015_aligned.csv', index=False)

🐧 Re-aligning with Valve-First Strategy...

--- Improved Valve Sanity Check ---
2022 Distance: -0.0030 ft
2007 Aligned:  -0.0030 ft
New Error:     0.0000 ft (Target: < 0.1)


In [14]:
v_22 = df_22[df_22['feature_type'].str.contains('Valve', na=False)]['distance'].iloc[0]
v_15_aligned = df_15_final[df_15_final['feature_type'].str.contains('Valve', na=False)]['aligned_dist'].iloc[0]

print(f"--- 2015 Sanity Check ---")
print(f"Accuracy: {abs(v_22 - v_15_aligned):.4f} ft error")

--- 2015 Sanity Check ---
Accuracy: 0.0000 ft error


In [15]:
def get_matching_pairs(df1, df2):
    # 1. Ensure we are looking for ANY metal loss
    # (Matches 'Internal Metal Loss', 'External', etc.)
    df1_ml = df1[df1['feature_type'].str.contains('Metal Loss|Loss', na=False, case=False)]
    df2_ml = df2[df2['feature_type'].str.contains('Metal Loss|Loss', na=False, case=False)]
    
    if len(df1_ml) == 0 or len(df2_ml) == 0:
        print("❌ One of the years has no 'Metal Loss' events labeled. Check feature_type names!")
        return pd.DataFrame()

    # 2. Create the Cost Matrix
    # We use a 20-foot tolerance because matching rust is harder than matching welds
    dist_matrix = np.abs(df1_ml['aligned_dist'].values[:, np.newaxis] - df2_ml['distance'].values)
    
    # 3. Hungarian Algorithm
    row_ind, col_ind = linear_sum_assignment(dist_matrix)
    
    matches = []
    for r, c in zip(row_ind, col_ind):
        # Increased tolerance to 20 feet to find those 'hidden' matches
        if dist_matrix[r, c] < 20.0: 
            matches.append({
                'id_2015': r,
                'location_22': df2_ml.iloc[c]['distance'],
                'dist_diff': dist_matrix[r, c],
                'depth_15': df1_ml.iloc[r]['depth'],
                'depth_22': df2_ml.iloc[c]['depth']
            })
    return pd.DataFrame(matches)

# Execute again
matched_df = get_matching_pairs(df_15_final, df_22)
print(f"✅ Pairing attempt finished. Found {len(matched_df)} matches!")

✅ Pairing attempt finished. Found 187 matches!


In [9]:
# Calculate Growth Rate
# 2022 depth minus 2015 depth, divided by the 7-year gap
matched_df['total_growth'] = matched_df['depth_22'] - matched_df['depth_15']
matched_df['growth_per_year'] = matched_df['total_growth'] / 7

# Identify the "Danger Spots"
# Spots growing faster than 2% depth per year are high priority
danger_spots = matched_df[matched_df['growth_per_year'] > 2.0]

print(f"--- Corrosion Analysis Summary ---")
print(f"Average Depth in 2015: {matched_df['depth_15'].mean():.2f}%")
print(f"Average Depth in 2022: {matched_df['depth_22'].mean():.2f}%")
print(f"Average Growth Rate:   {matched_df['growth_per_year'].mean():.4f}% per year")
print(f"🚨 DANGER: {len(danger_spots)} spots are growing at critical rates!")

--- Corrosion Analysis Summary ---
Average Depth in 2015: 17.73%
Average Depth in 2022: 18.67%
Average Growth Rate:   0.1352% per year
🚨 DANGER: 20 spots are growing at critical rates!


In [21]:
# Cross-Verification Script
def fuzzy_clock_match(c1, c2):
    try:
        # Helper to convert almost any format to "Minutes from 12:00"
        def to_minutes(val):
            val = str(val).strip()
            if ':' in val: # Handle "10:30"
                h, m = map(int, val.split(':')[:2])
                return h * 60 + m
            return float(val) * 60 # Handle "10.5"
        
        diff = abs(to_minutes(c1) - to_minutes(c2))
        return diff <= 30 # Allow 30 mins of "wobble"
    except:
        return False

# Re-verify with the Fuzzy Logic
verified_results = []
for _, match in matched_df.iterrows():
    row_15 = df_15_final[df_15_final['depth'] == match['depth_15']].iloc[0]
    row_22 = df_22[df_22['distance'] == match['location_22']].iloc[0]
    
    verified_results.append(fuzzy_clock_match(row_15['clock'], row_22['clock']))

matched_df['clock_verified'] = verified_results
accuracy = (matched_df['clock_verified'].sum() / len(matched_df)) * 100

print(f"--- 🛡️ RECOVERED PHASE 2 REPORT ---")
print(f"Clock Position Match Rate: {accuracy:.2f}%")
print(f"Verified Danger Spots:     {matched_df[(matched_df['clock_verified']) & (matched_df['growth_per_year'] > 2.0)].shape[0]} out of 20")

--- 🛡️ RECOVERED PHASE 2 REPORT ---
Clock Position Match Rate: 13.37%
Verified Danger Spots:     4 out of 20


In [23]:
# Create the Priority List
# We prioritize the 4 verified spots first, then the other high-growth spots
priority_list = matched_df.sort_values(by=['clock_verified', 'growth_per_year'], ascending=False).head(10)

# Format for the presentation
presentation_table = priority_list[['location_22', 'growth_per_year', 'clock_verified']]
presentation_table.columns = ['Distance (ft)', 'Growth Rate (%/yr)', 'Orientation Verified?']

print("TOP RISK SITES")
print(presentation_table.to_string(index=False))

TOP RISK SITES
 Distance (ft)  Growth Rate (%/yr)  Orientation Verified?
     45013.179            2.571429                   True
     44813.491            2.428571                   True
     15007.881            2.142857                   True
     29241.407            2.142857                   True
     44887.146            1.285714                   True
     45253.973            1.142857                   True
     45917.526            1.142857                   True
     43850.144            1.000000                   True
     43851.601            1.000000                   True
     56734.350            0.571429                   True
